In [1]:
%run Signed_fundamental_domain_Espinoza.ipynb
%run Intersect_lattice_with_cone.ipynb

HELPER AND SHINTANI FUNCTIONS

In [2]:
def get_decimals(z):
    if isinstance(z, float):
        return z
    return float(z.real()) + I*float(z.imag())

In [3]:
def compute_Bernoulli_Barnes(r, x, M = 5):
    """Return the 0,1,...,M-Bernoulli-Barnes polynomials of Bayad and Beck.
    Each entry should be a polynomial in a0, a1, a2, ... , a_(r-1) and in x (where r is Ruijsenaar's N)."""

    Berndict = dict([]) # indexed by k, with each value a function of a (always set w = 0) 
    
    ringofa = LaurentPolynomialRing(QQ, 'a', r)
    ringofa.inject_variables(verbose = False)
    base.<x> = PolynomialRing(ringofa)
    R.<t> = PowerSeriesRing(base, default_prec = M + 2)

    denominator = 1
    for aj in ringofa.gens():
        denominator *= exp(aj*t) - 1
    invertible = denominator/t^r
    fraction = exp(x*t)/invertible

    coeffs = fraction.coefficients()
    max_power = len(coeffs) - 1
    if max_power != M:
        print('Uh oh')
        print('The number of coefficients is', len(coeffs))
    for k in range(0, max_power + 1):
        Berndict[k] = factorial(k)*coeffs[k](0, ringofa.gens())

    return Berndict

In [4]:
def compute_Bernoullis(r, M = 5):
    """Return the 0,1,...,M-Bernoulli polynomials of Ruijsenaars evaluated at x = 0.
    Each entry should be a polynomial in a0, a1, a2, ... , a_(r-1) (where r is Ruijsenaar's N). 
    Expect r < M with M at most 18."""

    Berndict = dict([]) # indexed by k, with each value a function of a (always set w = 0) 
    
    ringofa = LaurentPolynomialRing(QQ, 'a', r)
    ringofa.inject_variables(verbose = False)
    base.<x> = PolynomialRing(ringofa)
    R.<t> = PowerSeriesRing(base, default_prec = M + 2)

    denominator = 1
    for aj in ringofa.gens():
        denominator *= exp(aj*t) - 1
    invertible = denominator/t^r
    fraction = exp(x*t)/invertible

    coeffs = fraction.coefficients()
    max_power = len(coeffs) - 1
    if max_power != M:
        print('Uh oh')
        print('The number of coefficients is', len(coeffs))
    for k in range(0, max_power + 1):
        Berndict[k] = factorial(k)*coeffs[k](0, ringofa.gens())

    return Berndict

In [5]:
def Barnes_zeta_at_0(r, a, w, Bern):
    """Calculate Barnes zeta function at s = 0.
    Expect w as a complex number with positive real part.
    Expect a as a tuple of r complex numbers (with positive real part).
    Expect Berndict to have length at least r + 1."""   
    
    Bernrsum = 0
    for k in range(0, r+1):
        Bernrsum += Bern[k]/factorial(k)*w^(r - k)/factorial(r - k)
    return Bernrsum*(-1)^r

    # ALTERNATE: using Bernr had similar time cost
    # Bernr = coeffs[r] of compute_Bernoullis(r, M) 
    # Barnes zeta = Bernr(w, a)*(-1)^r
    # M = len(Berndict.keys()) - 1

In [6]:
def get_permutations(r):
    """Return a list of all permutations of {0, 1, ... , r} whose sum equals r."""
    # TODO: dynamic programming would make this more efficient
    perms = []
    for pieces in Tuples(range(0, r + 1), r).list(): # Tuples are ordered, with repitition
        if sum(pieces) == r:
            perms.append(pieces)
    return perms

In [7]:
def Shintani_zeta_at_0(r, A, x, permutations_of_r):
    """Calculate Shintani zeta function at s = 0.
    Expect x as an r-tuple of positive real numbers.
    Expect A as a list of r-tuples of complex numbers (with positive real part)."""
    sum = 0
    for k in range(0, len(A)):
        column = A[k]
        for indices in permutations_of_r:
            prod = 1
            for j in range(0, r):
                kj = indices[j]
                prod *= bernoulli_polynomial(x[j], kj)/factorial(kj)*column[j]^(kj - 1)
            sum += prod
    return (-1)^r*sum

In [8]:
def Shintani_Barnes_zeta_at_0(r, a, x, permutations_of_r):
    """Calculate Barnes zeta function at s = 0 by regarding as Shintani zeta function.
    Expect x as an r-tuple of positive real numbers.
    Expect a as an r-tuple of complex numbers (with positive real part)."""
    return Shintani_zeta_at_0(r, [a], x, permutations_of_r)

RUIJSENAARS CALCULATION OF LOG GAMMA

In [9]:
def exact_Ruij_log_gamma(r, a, w, Bern, embedding, rotation):
    """Calculate Ruijsenaars log_gamma_r with given Bern list.
    Expect w as a complex number with (large) positive real part.
    Expect a as a tuple of r complex numbers with positive real part."""

    import sympy as sym
    M = len(Bern) - 1    

    firstSum = 0
    secondSum = 0
    # TODO: can I make the log exact here?
    # exponent = sym.simplify(log(rotation))
    # loggamma = -(exponent + log(embedding(w)))*rotation^(2*r)*embedding(Barnes_zeta_at_0(r, a, w, Bern))
    loggamma = -log(get_decimals(rotation)*embedding(w))*rotation^(2*r)*embedding(Barnes_zeta_at_0(r, a, w, Bern))

    # note: if used loggamma - loggamma2, then type "Add" and has no attribute real()...?
    # most of the differences were magnitude 10^(-15), but some were not in 2*pi*I*Z...
    # print('difference =', get_decimals(loggamma2 - loggamma).n(digits = 5))
        
    # compute first sum
    for k in range(0, r): 
        innersum = 0
        for j in range(1, r - k + 1):
            innersum += j^(-1)
        firstSum += Bern[k]/factorial(k)*w^(r - k)/factorial(r - k)*innersum
    # compute second sum
    # these terms are the same as the series obtained by termwise integration 
    for k in range(r + 1, M + 1):
        secondSum += (-1)^k*Bern[k]/factorial(k)*w^(r-k)*factorial(k-r-1)
    # add to log gamma
    loggamma += rotation^(2*r)*embedding(firstSum + (-1)^r*secondSum)
    
    return loggamma

In [10]:
def Ruij_log_gamma(r, a, w, Bern):
    """Calculate Ruijsenaars log_gamma_r with given Berndict.
    Expect w as a complex number with (large) positive real part.
    Expect a as a tuple of r complex numbers with positive real part."""
   
    M = len(Bern) - 1

    # TODO: check what's happening in this code when M is large?
    
    firstSum = 0
    secondSum = 0
    loggamma = -log(w)*Barnes_zeta_at_0(r, a, w, Bern)
    
    # compute and add first sum
    for k in range(0, r): 
        innersum = 0
        for j in range(1, r - k + 1):
            innersum += j^(-1)
        firstSum += Bern[k]/factorial(k)*w^(r - k)/factorial(r - k)*innersum
    loggamma += (-1)^r*firstSum
    
    # compute and add second sum
    # these terms are the same as the series obtained by termwise integration 
    for k in range(r + 1, M + 1):
        secondSum += (-1)^k*Bern[k]/factorial(k)*w^(r-k)*factorial(k-r-1)
    loggamma += secondSum
    
    return get_decimals(loggamma) 

In [11]:
def Ruij_log_gamma_with_error(r, a, w, Bern, threshold = 2):
    """Calculate Ruijsenaars log_gamma_r with integral bounds determined by threshold.
    Expect w as a complex number with positive real part.
    Expect a as a tuple of r complex numbers (with positive real part)."""
    
    loggamma = Ruij_log_gamma(r, a, w, Bern, printing = False)
    # compute and add error (integral)
    remainder = Ruij_integral_error(r, a, w, Bern, threshold, printing = False)
    loggamma += remainder
    
    # compute and compare alternate error (series)
    # series = series_error(r, a, w, M)
    # print('series error =', series)
    # difference = remainder - series
    # print('difference in errors =', get_decimals(difference))
    
    return get_decimals(loggamma)

In [12]:
def Ruij_integral_error(r, a, w, Bern, threshold, printing = False):
    """Approximate the error term (infinite line integral) of Ruijsenaar's log_gamma asymptotics."""
    
    M = len(Bern) - 1
    u = var('u')
    product = 1
    for j in range(1,r + 1):
        product *= (1 - e^(-a[j - 1]*u))^(-1)
    sum = 0
    for m in range(0,M + 1):
        sum += (-1)^m*Bern[m]/factorial(m)*u^(m - r)
    integrand = e^(-w*u)/u*(product - sum)
    
    precision = 1_000
    CCb = ComplexBallField(precision)
    
    f = fast_callable(integrand,vars=[u],domain=CCb)
    remainder = CC(CCb.integral(lambda u,_: f(u), CCb(1/10^(threshold)), CCb(10^(5))))
    
    if printing:
        print('integral error =', get_decimals(remainder))

    # compute and print a left tail
    # left_tail = CC(CCb.integral(lambda u,_: f(u), CCb(1/10^(30)),CCb(1/10^(threshold))))
    # print('at (M, threshold) = ', (M, threshold), 'left tail =', get_decimals(left_tail))
    
    return remainder

In [13]:
def series_error(r, a, w): # gave huge results, so cannot swap series and integral 
    """Compute hypothetical series expression for the error in log gamma approximation."""
    upper = 18
    Berndict = compute_Bernoullis(r, upper)
    Bern = [Berndict[m](a) for m in range(0,upper + 1)]
    
    series = 0
    for k in range(r + 1, upper + 1):
        term = (-1)^k*Bern[k]/factorial(k)*w^(r-k)*factorial(k-r-1)
        # print('index k =', k, '\t series term =', get_decimals(term)) 
        if k >= M + 1:
            series += term
    return get_decimals(series)

RECURSIVE CALCULATION OF LOG GAMMA

In [14]:
from dataclasses import dataclass
from functools import cmp_to_key
import matplotlib as mpl
@dataclass
class logGammaApproxData:
    """Data of approximation of log_gamma function for a cone."""
    def __init__(self, cone_gens, columnofA, threshold = 10, M = 15):
        """Compute and store a_list as a sorted list of pairs (a cone gen paired with its embedding in columnofA)."""
        # setup/store basic info
        self.threshold = threshold
        self.r = len(cone_gens)
        # initialize Bernoulli dictionaries
        Berndicts = dict([])
        for k in range(1, self.r + 1): # compute required Bernoullis
            Berndicts[k] = compute_Bernoullis(k, M)
        self.Berndicts = Berndicts # TODO: memoize these results for efficiency

        # sort cone gen and (rotated) column by abs_value
        a_list = list(zip(cone_gens, columnofA)) # pair cone gens with their rotated embeddings       
        abs_val = lambda pair : -abs(pair[1])
        a_list.sort(key = abs_val) # sort
        # store sorted cone_gens and columnofA
        self.cone_gens = [pair[0] for pair in a_list]
        self.rotated_gens = [pair[1] for pair in a_list]
        # if self.r == 3:
        #     print('largest/smallest =', (self.rotated_gens[0]/self.rotated_gens[2]).n(digits = 4))
        # place to store all results
        self.all_results = dict([]) # will be indexed by xvecs
        
    
    def print(self):
        print('threshold =', self.threshold)
        print('r =', self.r)
        print('a_list =', self.a_list)
        print('len(Berndicts) =', len(self.Berndicts))

In [15]:
@dataclass
class logGammaResults:
    """Data of results for log_gamma of a_list and xvec."""
    def __init__(self):    
        # data for base case r = 0
        self.exact_product = 1
        self.approx_product = 1
        self.num_terms = 0
        # data for base case abs(w) large
        self.exact_tails = []
        self.approx_tails = []

In [16]:
def recursive_log_gamma(cone_approx_data, list_xvecs, embedding = lambda x: x):
    """Recursively calculate log_gamma_r with given approximation data.
    Store results in approximation data.
    Expect xvecs as a list of r-tuples of rational numbers."""
    # setup recursion parameters
    r = cone_approx_data.r
    ind_a = [1,]*r
    ind_w = [0,]*r
    
    for xvec in list_xvecs:
        # setup approximation data
        results = logGammaResults()
        cone_approx_data.all_results[str(xvec)] = results
        
        exact_w = dot_product(cone_approx_data.cone_gens, xvec)
        approx_w = dot_product(cone_approx_data.rotated_gens, xvec)

        # recursively calculate and store data for log_gamma
        log_gamma_recurse(cone_approx_data, results, r, ind_a, exact_w, approx_w, embedding)

In [17]:
def log_gamma_recurse(cone_approx_data, results, r, ind_a, exact_w, approx_w, embedding):
    """Recursive function for calculation of log_gamma_r with given approximation data.
    Expect w as a complex number with positive real part.
    Expect ind_a as a tuple of 0's and 1's, with r ones."""
  
    cone_gens = cone_approx_data.cone_gens
    rotated_gens = cone_approx_data.rotated_gens
    
    if r == 0: # base case 1
        results.exact_product *= 1/exact_w
        results.approx_product *= 1/approx_w
        results.num_terms += 1
        return
        
    if abs(approx_w) > cone_approx_data.threshold: # base case 2: if magnitude of w is large enough
        results.exact_tails.append((r, ind_a, exact_w)) # record exact end of tree
        results.approx_tails.append((r, ind_a, approx_w)) # record approx end of tree 

        diff = abs(approx_w) - abs(embedding(exact_w))
        if not diff < 10^(-10):
            print('Uh oh, recursion not working? Check this base case.')
        return
        
    # otherwise, recurse
    index = ind_a.index(1) # use first available cone generator
    new_ind_a = ind_a.copy()
    new_ind_a[index] = 0 # record that we used the cone generator
    # update w
    exact_translate = cone_gens[index]
    approx_translate = rotated_gens[index]
    
    log_gamma_recurse(cone_approx_data, results, r, ind_a, exact_w + exact_translate, approx_w + approx_translate, embedding) # move toward base case 2
    log_gamma_recurse(cone_approx_data, results, r - 1, new_ind_a, exact_w, approx_w, embedding) # move toward base case 1

In [18]:
def log_gamma_from_data(cone_approx_data, xvec, rotation = 1, embedding = lambda x: x):
    """Compute log_gamma from stored results."""

    import sympy as sym
       
    results = cone_approx_data.all_results[str(xvec)]

    # data for base case r = 0
    exact_product = results.exact_product
    num_terms = results.num_terms
    approx_product = results.approx_product
    # data for base case abs(w) large
    exact_tails = results.exact_tails
    approx_tails = results.approx_tails
        
    # calculate sum with approximate results
    approx_sum = 0
    approx_sum += log(approx_product) # add results from r = 0
    for (k, ind_a, approx_w) in approx_tails: # add results from approx tails
        a = []
        for (included, aj) in zip(ind_a, cone_approx_data.rotated_gens):
            if included:
                a.append(aj)
        r = len(a)
        Bern = [B(a) for B in cone_approx_data.Berndicts[r].values()]

        # TODO: make sure that len(Bern) changes as M changes
        print("length of Bern = ", len(Bern))
        
        approx_sum += Ruij_log_gamma(k, a, approx_w, Bern)

    return approx_sum
    
    # calculate sum with exact results

    exponent = sym.simplify(log(rotation))
    exact_sum = 0
    exact_sum += log(embedding(exact_product)) + (-num_terms)*exponent # add results from r = 0
    for (k, ind_a, exact_w) in exact_tails: # exact results
        a = []
        for (included, aj) in zip(ind_a, cone_approx_data.cone_gens):
            if included:
                a.append(aj)
        Bern = [B(a) for B in cone_approx_data.Berndicts[len(a)].values()]
        # if embedding(domain.field.theta).imag() != 0:
        #     pdb.set_trace()
        exact_sum += exact_Ruij_log_gamma(k, a, exact_w, Bern, embedding, rotation)
    
    return exact_sum

VERIFICATION functions

Alexanian and Kuznetsov formula functions

In [19]:
def gamma_2_AK(w1, w2, z, precision = 20):
    """Expect w1 and w2 as complex numbers aways from (-inf, 0] with w1 = w2.
    Expect z as a complex number. 
    Calculate via the formula of Barnes, 1904, as reported in Alexanian and Kuznetsov, 2023."""
       
    prod = 1
    prod *= (2*pi)^(z/(2*w1))
    exponent = 0
    exponent += -z^2/(2*w1*w2)
    exponent += z*(w1 + w2)/(2*w1*w2)
    exponent += -1
    prod *= w2^exponent

    # import pdb
    # pdb.set_trace()
        
    math_z1 = mathematica(z/w1)
    Barnes = math_z1.BarnesG()
    # Barnes = mathematica('N[BarnesG[' + str(z1) + '],' + str(precision) + ']')
    prod *= Barnes.sage()^(-1)

    return get_decimals(prod)

NUMERICAL EXPLORATIONS scripts

In [20]:
def graph_log_gamma(dimn = 2, count = 100, debugging = False):
    """Expect dimn to be 1 or 2.
    Perform count tests of the log_gamma function using formulas from the literature.
    Graph the differences (between computed and actual results) as a function of input magnitude."""  
    import random
    import sympy
    
    abs_val_zs = []
    abs_val_results = []

    CC = ComplexField(100)
    
    for n in range(0, count):  # perform count-many trials
        # pick random test value z
        x = random.uniform(0, 10)
        y = random.uniform(-10, 10)
        w = CC(x, y)
        xvec = []
        for n in range(0,dimn):
            p = randint(1, 6)
            q = randint(1, p + 1)
            xvec.append(p/q)
        z = dot_product([w]*dimn, xvec)
        
        # compute expected result from literature
        resultLit = 0
        if dimn == 1:
            resultLit = gamma(z)/RR(sqrt(2*pi))
        if dimn == 2:
            resultLit = gamma_2_AK(w, w, z)
        
        abs_val_zs.append(abs(z))
        abs_val_results.append(abs(resultLit))

    # GRAPH RESULTS
    
    import matplotlib as mpl
    import matplotlib.pyplot as plt

    x_axis = 'Abs value of z'
    y_axis = 'Abs value of log gamma ' + str(dimn)

    title = x_axis + ' vs. ' + y_axis
    name = 'Gamma' + str(dimn) + '_Graphs/'
    name += title
        
    fig, ax = plt.subplots(figsize = (9, 6))
    ax.scatter(abs_val_zs, abs_val_results, s = 12)
    ax.set_yscale("log")

    # import sklearn
    # from sklearn.metrics import r2_score
    # rvalue = r2_score(abs_val_zs, abs_val_results)\
    # ax.annotate("r-squared = {:.3f}".format(rvalue), (0, 1))
    
    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)       
    plt.savefig(name + '.png')
    plt.clf()

In [21]:
# graph_log_gamma(dimn = 2)

<Figure size 900x600 with 0 Axes>

In [67]:
def test_log_gamma(dimn = 2, vary_M = False, fixed = 50, debugging = False):
    """Expect dimn to be 1 or 2. Compute approximate and expected values of log_gamma functions of given dimn.
    If vary_M = False, use the given fixed value for M, and graph different threshold values in different colors.
    If vary_M = True, use the given fixed value for threshold, and graph different M values in different colors."""  
    import random
    import sympy
    
    count = 30
    color_dict = dict([])
    differences = dict([])
    abs_val_zs = dict([])
    ratios = dict([])
    percentages = dict([])
    abs_val_results = dict([])
    
    color_values = ['red', 'orange', 'green', 'blue', 'purple']
    M_values = [5, 10, 25, 50]        
    threshold_values = [5, 10, 20, 40, 80]

    if vary_M:
        threshold_values = [fixed]
        for (M, color) in zip(M_values, color_values):
            color_dict[M] = color
            differences[M] = []
            abs_val_zs[M] = []
            ratios[M] = []
            percentages[M] = []
            abs_val_results[M] = []
    else:
        M_values = [fixed] 
        threshold_color_dict = dict([])
        for (threshold, color) in zip(threshold_values, color_values):
            color_dict[threshold] = color
            differences[threshold] = []
            abs_val_zs[threshold] = []
            ratios[threshold] = []
            percentages[threshold] = []
            abs_val_results[threshold] = []
    
    CC = ComplexField(100)
    
    for n in range(0, count):  # perform count-many trials
        # pick random test value z
        x = random.uniform(0, 1)
        y = random.uniform(-1, 1)
        w = CC(x, y)
        xvec = []
        for n in range(0,dimn):
            p = randint(1, 6)
            q = randint(1, p + 1)
            xvec.append(p/q)
        z = dot_product([w]*dimn, xvec)
        
        # compute expected result from literature
        if dimn == 1:
            resultLit = gamma(z)/RR(sqrt(2*pi))
        if dimn == 2:
            resultLit = gamma_2_AK(w, w, z)

        # if abs(z) > 1:
        #     continue
        
        # approximate result with my Ruijsenaars-based code
        for M in M_values:
            for threshold in threshold_values:
                if vary_M:
                    parameter = M
                else:
                    parameter = threshold
        
                if dimn == 1:               
                    approx_data = logGammaApproxData([1], [1], M = M, threshold = threshold)
                    recursive_log_gamma(approx_data, [[z]])  # store recursive calc in approx_data
                    result = exp(log_gamma_from_data(approx_data, [z]))  # get recursive calc from approx_data
                if dimn == 2:
                    approx_data = logGammaApproxData([w, w], [w, w], M = M, threshold = threshold)
                    recursive_log_gamma(approx_data, [xvec])  # store recursive calc in approx_data
                    result = exp(get_decimals(log_gamma_from_data(approx_data, xvec)))  # get recursive calc from approx_data
        
                if debugging:
                    import pdb 
                    pdb.set_trace()
                    
                difference = abs(resultLit - result)
                if difference < 10^(-20): # TODO: update if vary precision
                    difference = 10^(-20)  
                elif difference > 10^10:
                    difference = 10^10
                differences[parameter].append(log(difference, 10))
                
                percentage = 100*difference/abs(resultLit)
                if percentage > 1:
                    percentage = 1
                percentages[parameter].append(percentage)
                
                ratio = abs(resultLit/result)
                ratios[parameter].append(ratio)
                
                abs_val_zs[parameter].append(abs(z))
                abs_val_results[parameter].append(abs(resultLit))

    data_dict = dict([])
    if vary_M:
        data_dict['varying_variable'] = M_values
    else:
        data_dict['varying_variable'] = threshold_values
    data_dict['Abs value of z'] = abs_val_zs
    data_dict['Log of difference'] = differences
    data_dict['Abs value of result'] = abs_val_results
    data_dict['Percentage error in difference'] = percentages
    data_dict['color_dict'] = color_dict
    
    # GRAPH RESULTS OF COMPUTATIONS
    
    import matplotlib as mpl
    import matplotlib.pyplot as plt

    plot_difference = True
    plot_percentage = False
    plot_diff_percent = False

    if vary_M:
        title = 'threshold = ' + str(threshold)
    else:
        title = 'M = ' + str(M)

    if plot_difference:
        x_axis = 'Abs value of z'
        y_axis = 'Log of difference'

        name = 'Gamma' + str(dimn) + '_Graphs/'
        name += x_axis + ' vs. ' + y_axis + '/'
        name += title
        
        plot_data(x_axis, y_axis, data_dict, title, plt, name)
        
    if plot_percentage:
        x_axis = 'Abs value of z'
        y_axis = 'Percentage error in difference'
        
        name = 'Gamma' + str(dimn) + '_Graphs/'
        name += x_axis + ' vs. ' + y_axis + '/'
        name += title
        
        plot_data(x_axis, y_axis, data_dict, title, plt, name)
        
    if plot_diff_percent:
        x_axis = 'Log of difference'
        y_axis = 'Percentage error in difference' 
        
        name = 'Gamma' + str(dimn) + '_Graphs/'
        name += x_axis + ' vs. ' + y_axis + '/'
        name += title
        
        plot_data(x_axis, y_axis, data_dict, title, plt, name)

In [68]:
def plot_data(x_axis, y_axis, data_dict, title, plt, name):
    """Helper function for plotting results of log gamma tests."""
    values = data_dict['varying_variable']
    for value in values:
        
        x_data = data_dict[x_axis][value]
        y_data = data_dict[y_axis][value]
        color = data_dict['color_dict'][value]
        plt.scatter(x_data, y_data, color = color, s = 12, alpha = 0.5)
        
    plt.title(title)
    plt.xlabel(x_axis)
    plt.ylabel(y_axis)

    if 'M' in title:
        plt.legend(values, title = 'threshold')
    elif 'threshold' in title:
        plt.legend(values, title = 'M')
        
    plt.savefig(name)
    plt.clf()

In [70]:
# test_log_gamma(dimn = 1, vary_M = True, fixed = 5, debugging = True)
# test_log_gamma(dimn = 2, vary_M = True, fixed = 10, debugging = False)
# test_log_gamma(dimn = 1, vary_M = True, fixed = 20, debugging = True)
# test_log_gamma(dimn = 1, vary_M = True, fixed = 40, debugging = True)
# test_log_gamma(dimn = 1, vary_M = True, fixed = 80, debugging = True)

<Figure size 640x480 with 0 Axes>

In [25]:
# test_log_gamma(dimn = 1, vary_M = False, fixed = 5, debugging = False)
# test_log_gamma(dimn = 1, vary_M = False, fixed = 10, debugging = False)
# test_log_gamma(dimn = 1, vary_M = False, fixed = 25, debugging = False)
# test_log_gamma(dimn = 1, vary_M = False, fixed = 50, debugging = False)
# test_log_gamma(dimn = 1, vary_M = False, fixed = 100, debugging = False)

For log gamma 1, I visually inspected the graphs and determined that threshold = 10 and M = 50 performs best.

Notes when stepping through the code with threshold = 10 (or 80) and M varying:
- for extremely small gamma 1 (small outputs), increasing from M = 5 to M = 10 to M = 25 the accuracy improves at each step...
- - but no change in accuracy at M = 50 or M = 100...
- for large gamma 1 (large outputs), increasing from M = 5 to M = 10 does still give improvement in number of significant digits...
- - but no change in accuracy at M = 25 or above...! Why isn't M = 25 still improving??

In the previous box, I'm only really considering the magnitude of resultLit but not fully correlated with size of input...
- could it be that one of {size of input, size of output} impacts the accuracy (and relevant M, threshold) while the other doesn't?
- could it be that the additional contribution to the approximation that comes from increasing $M$ is so small that it rounds to zero?
- next step: step through code and investigate how the accuracy changes as $M$ increases for small inputs and large inputs?

In [26]:
def check_recurrence_relations(r, a, w, threshold = 10):
    """Check log_gamma_r difference equation with error of parameter M. 
    Expect w as a complex number with positive real part.
    Expect a as a tuple of r complex numbers (with positive real part)."""
    
    approx_data = logGammaApproxData(a, M = 18, threshold = threshold)
    approx_data_down = logGammaApproxData(a[:-1], M = 18, threshold = threshold)
    
    LHS = recursive_log_gamma(approx_data, w)
    RHS = recursive_log_gamma(approx_data, w + a[-1])
    RHS += recursive_log_gamma(approx_data_down, w + a[-1])
    
    difference = LHS - RHS
    return difference

In [27]:
def check_diag_mult_log_gamma(r, n):
    """Check my lemma 3.1 on a random r by n matrix."""
    import random
    # construct random A and U so that A and AU have positive real parts
    A = []
    AU = []
    rotations = []
    for k in range(0, n): # construct matrices by column
        column = []
        rot_column = []
        arguments = []  
        for j in range(0, r):
            x = random.uniform(0, 100)
            y = random.uniform(-10, 10)
            z = x + I*y
            column.append(z)
        rotation = get_decimals(compute_rotation(column))
        rotations.append(rotation)
        rot_column = [z*rotation for z in column]
        for z in rot_column:
            if not z.real() > 0:
                print('Rotation did not work as expected :( ')
        A.append(column)
        AU.append(rot_column)

    # compute difference of log gammas and Barnes zeta
    x_vec = []
    for j in range(0, r):
        x = random.uniform(0, 1)
        x_vec.append(x)
        
    for k in range(0, n):
        column = A[k]
        approx_data = logGammaApproxData(column, M = 15, threshold = 10)
        rot_column = AU[k]
        rot_approx_data = logGammaApproxData(rot_column, M = 15, threshold = 10)

        w = dot_product(x_vec, column)
        lg = recursive_log_gamma(approx_data, w)
        rot_w = dot_product(x_vec, rot_column)
        rot_lg = recursive_log_gamma(rot_approx_data, rot_w)

        RHS = lg - rot_lg
        LHS = log(rotations[k])*Shintani_Barnes_zeta_at_0(r, column, x_vec, get_permutations(r))

        print('k =', k)
        print('RHS =', RHS)
        print('LHS =', LHS)
        print()

NOTES:
- integral threshold must be at least 2, while 3 is better, and higher values grant diminishing returns (but still more accurate)
    - the upper bound in the integral computation doesn't matter much (past a certain sufficiently large value)
    - meanwhile, the lower bound has a substantial (even unpredictable effect)
- I get unexpected (seemingly erratic) results when M is too small (4 or lower) or M is too large (14 or greater)
- the difference equations give at least some measure of accuracy
    - I introduced typos at various parts of the log_gamma code, and found the differences to be large/random
    - the recursive approach yields much smaller differences when checking the recurrence relations
        - maybe this is an indicator of accuracy, or maybe just a reflection of the fact that it leverages the relations themselves